<a href="https://colab.research.google.com/github/masies/CRA/blob/main/evaluation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
! pip install -q torch==1.4.0 -f https://download.pytorch.org/whl/cu101/torch_stable.html
! pip install -q transformers==3.5.0 fast-trees
! git clone -q https://github.com/microsoft/CodeXGLUE.git

from os import walk
import os
import regex

from google.colab import auth
auth.authenticate_user()
project_id = 'helical-loop-303918'
bucket_name = 'code_review_automation'
!gcloud config set project {project_id}

     |████████████████████████████████| 753.4MB 21kB/s 
ERROR: torchvision 0.9.1+cu101 has requirement torch==1.8.1, but you'll have torch 1.4.0 which is incompatible.
ERROR: torchtext 0.9.1 has requirement torch==1.8.1, but you'll have torch 1.4.0 which is incompatible.
     |████████████████████████████████| 1.3MB 3.8MB/s 
     |████████████████████████████████| 2.9MB 47.6MB/s 
     |████████████████████████████████| 1.1MB 44.2MB/s 
     |████████████████████████████████| 901kB 38.4MB/s 
     |████████████████████████████████| 163kB 43.6MB/s 
     |████████████████████████████████| 112kB 48.6MB/s 
     |████████████████████████████████| 71kB 7.3MB/s 
Updated property [core/project].


In [4]:
!mkdir ./code_code
!mkdir ./codeANDcomment_code
!mkdir ./marked_code
!mkdir ./code_comment
!mkdir ./mixture



# !gsutil -m cp -r gs://code_review_automation/fine_tuning/HP_tuning/big_model/code_code/validation_eval ./code_code
# !gsutil -m cp -r gs://code_review_automation/fine_tuning/HP_tuning/big_model/codeANDcomment_code/validation_eval ./codeANDcomment_code
# !gsutil -m cp -r gs://code_review_automation/fine_tuning/HP_tuning/big_model/code_comment/validation_eval ./code_comment
# !gsutil -m cp -r gs://code_review_automation/fine_tuning/HP_tuning/big_model/marked_code/validation_eval ./marked_code
!gsutil -m cp -r gs://code_review_automation/fine_tuning/HP_tuning/big_model/mixture/slanted/validation_eval ./mixture

mkdir: cannot create directory ‘./code_code’: File exists
mkdir: cannot create directory ‘./codeANDcomment_code’: File exists
mkdir: cannot create directory ‘./marked_code’: File exists
mkdir: cannot create directory ‘./code_comment’: File exists
mkdir: cannot create directory ‘./mixture’: File exists
Copying gs://code_review_automation/fine_tuning/HP_tuning/big_model/mixture/slanted/validation_eval/codeANDcomment_code_205000_predictions...
Copying gs://code_review_automation/fine_tuning/HP_tuning/big_model/mixture/slanted/validation_eval/codeANDcomment_code_210000_predictions...
Copying gs://code_review_automation/fine_tuning/HP_tuning/big_model/mixture/slanted/validation_eval/codeANDcomment_code_215000_predictions...
Copying gs://code_review_automation/fine_tuning/HP_tuning/big_model/mixture/slanted/validation_eval/codeANDcomment_code_220000_predictions...
Copying gs://code_review_automation/fine_tuning/HP_tuning/big_model/mixture/slanted/validation_eval/codeANDcomment_code_225000_pr

In [6]:
def eval(val_folder, tasks):
  for task in tasks:
    print(task)
    _, _, filenames = next(walk(val_folder))
    prediction_paths = [os.path.join(val_folder, f) for f in filenames if f.startswith(task) and f.endswith("predictions")]
    target_path = [os.path.join(val_folder, f) for f in filenames if f.startswith(task) and f.endswith("targets")][0]

    with open(target_path) as f:
        targets = [item.strip() for item in f.readlines()]

    for path in sorted(prediction_paths):
        step = regex.findall( task + "_(\d+)_predictions", path)[0]
        with open(path) as f:
          predictions = [item.strip() for item in f.readlines()]
        
        perfect_predictions = 0
        almost_perfect_predictions = 0
        for pred, target in zip(predictions, targets):
          if ' '.join(pred.strip().split()) == ' '.join(target.strip().split()):
            perfect_predictions += 1
          elif ''.join(pred.strip().split()) == ''.join(target.strip().split()):
            almost_perfect_predictions += 1
            index = next((i for i, (char1, char2) in enumerate(zip(pred, target)) if char1 != char2), None)
            
        print(step + " - PP: ", round(perfect_predictions/len(predictions)*100,2), " | APP: ", round((perfect_predictions + almost_perfect_predictions)/len(predictions)*100,2))
        print()

# eval('mixture/validation_eval', ["code_code", "codeANDcomment_code", "code_comment"])
# eval('code_code/validation_eval', ["code_code"])
# eval('codeANDcomment_code/validation_eval', ["codeANDcomment_code"])
# eval('code_comment/validation_eval', ["code_comment"])
eval('marked_code/validation_eval', ["marked_code"])
      


marked_code
210000 - PP:  2.04  | APP:  2.09

220000 - PP:  3.18  | APP:  3.29

230000 - PP:  4.05  | APP:  4.21

240000 - PP:  4.9  | APP:  5.06

250000 - PP:  5.49  | APP:  5.69

260000 - PP:  5.69  | APP:  5.88

270000 - PP:  6.03  | APP:  6.27

280000 - PP:  6.35  | APP:  6.56

290000 - PP:  6.31  | APP:  6.49

300000 - PP:  6.38  | APP:  6.58

310000 - PP:  6.7  | APP:  6.88

320000 - PP:  6.69  | APP:  6.92

330000 - PP:  6.64  | APP:  6.9

340000 - PP:  6.81  | APP:  7.03

350000 - PP:  6.72  | APP:  6.96

360000 - PP:  6.64  | APP:  6.87

370000 - PP:  6.81  | APP:  7.06

380000 - PP:  6.69  | APP:  6.91

390000 - PP:  6.96  | APP:  7.21

400000 - PP:  6.84  | APP:  7.12



In [ ]:
# def codeBleu(pred, target):
#   with open("pred.txt", "w") as f:
#     f.write(pred)
#   with open("tar.txt", "w") as f:
#     f.write(target)
#   try :
#     a = !cd /content/CodeXGLUE/Code-Code/code-to-code-trans/evaluator/CodeBLEU/ && python calc_code_bleu.py --refs ./../../../../../pred.txt --hyp ./../../../../../tar.txt --lang java --params 0.25,0.25,0.25,0.25
#     codeBleu = float(a[1].replace("CodeBLEU score:  ",""))
#     print(codeBleu)
#   except Exception as e:
#     codeBleu = 0

In [46]:
!rm -rf ./test
!mkdir test
!gsutil -m cp -r gs://code_review_automation/fine_tuning/HP_tuning/big_model/marked_code ./test

Copying gs://code_review_automation/fine_tuning/HP_tuning/big_model/marked_code/checkpoint...
Copying gs://code_review_automation/fine_tuning/HP_tuning/big_model/marked_code/events.out.tfevents.1618786617.fce44e709e33...
Copying gs://code_review_automation/fine_tuning/HP_tuning/big_model/marked_code/model.ckpt-200000.data-00000-of-00002...
Copying gs://code_review_automation/fine_tuning/HP_tuning/big_model/marked_code/events.out.tfevents.1618830521.61f7dfaf2b92...
Copying gs://code_review_automation/fine_tuning/HP_tuning/big_model/marked_code/model.ckpt-200000.data-00001-of-00002...
Copying gs://code_review_automation/fine_tuning/HP_tuning/big_model/marked_code/model.ckpt-200000.index...
Copying gs://code_review_automation/fine_tuning/HP_tuning/big_model/marked_code/model.ckpt-200000.meta...
Copying gs://code_review_automation/fine_tuning/HP_tuning/big_model/marked_code/graph.pbtxt...
Copying gs://code_review_automation/fine_tuning/HP_tuning/big_model/marked_code/model.ckpt-205000.data

In [49]:
checkpoints = !find ./test -name "model.ckpt-*.index" -print0 | xargs -0 ls
checkpoints = [x.replace("./test/marked_code/model.ckpt-","").replace(".index","") for x in checkpoints]
checkpoints

['200000',
 '205000',
 '205100',
 '210000',
 '210200',
 '215000',
 '215300',
 '220000',
 '220400',
 '225000',
 '225500',
 '230000',
 '230600',
 '235000',
 '235700',
 '240000',
 '240800',
 '245000',
 '245900',
 '250000',
 '251000',
 '255000',
 '256100',
 '260000',
 '261200',
 '265000',
 '266300',
 '270000',
 '271400',
 '275000',
 '276500',
 '280000',
 '281600',
 '285000',
 '286700',
 '290000',
 '291800',
 '295000',
 '296900',
 '300000',
 '302000',
 '305000',
 '307100',
 '310000',
 '312200',
 '315000',
 '317300',
 '320000',
 '322400',
 '325000',
 '327500',
 '330000',
 '332600',
 '335000',
 '340000',
 '340100',
 '345000',
 '345200',
 '350000',
 '350300',
 '355000',
 '355400',
 '360000',
 '360500',
 '365000',
 '365600',
 '370000',
 '370700',
 '375000',
 '375800',
 '380000',
 '380900']